In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.utils import np_utils

% matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

/Users/naman/miniconda3/envs/kaggle/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Loading Data

In [2]:
from mnist import MNIST
mndata = MNIST('./mnist/', gz=True)
images, labels = mndata.load_training()

In [3]:
def normalize(images):
    for i in range(len(images)):
        images[i] = images[i]/np.linalg.norm(images[i])
    return images
images = normalize(images)

### Get Test, Train, Validation Set

In [4]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [7]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

### Buid Model

In [14]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))                            
#model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.2))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
history = model.fit(X_train,y_train,epochs=50, batch_size=128,validation_data=(X_val, y_val))

Train on 38400 samples, validate on 9600 samples
Epoch 1/50
38400/38400 [==============================] - 9s 236us/step - loss: 2.8606e-05 - acc: 1.0000 - val_loss: 0.1096 - val_acc: 0.9808
Epoch 2/50
38400/38400 [==============================] - 10s 249us/step - loss: 2.4493e-05 - acc: 1.0000 - val_loss: 0.1104 - val_acc: 0.9814
Epoch 3/50
38400/38400 [==============================] - 9s 244us/step - loss: 2.1064e-05 - acc: 1.0000 - val_loss: 0.1110 - val_acc: 0.9812
Epoch 4/50
38400/38400 [==============================] - 9s 227us/step - loss: 1.8029e-05 - acc: 1.0000 - val_loss: 0.1126 - val_acc: 0.9807
Epoch 5/50
38400/38400 [==============================] - 9s 235us/step - loss: 1.5587e-05 - acc: 1.0000 - val_loss: 0.1131 - val_acc: 0.9812
Epoch 6/50
38400/38400 [==============================] - 9s 233us/step - loss: 1.3386e-05 - acc: 1.0000 - val_loss: 0.1143 - val_acc: 0.9808
Epoch 7/50
38400/38400 [==============================] - 9s 226us/step - loss: 1.1593e-05 - acc: 

In [16]:
model.evaluate(X_test,y_test,verbose=1)

12000/12000 [==============================] - 1s 109us/step


[0.11215791929579287, 0.9829166666666667]